### Sentiment Analysis in Social Media
This solution provides a basic framework for sentiment analysis on social media posts using different machine learning algorithms and an interactive UI for real-time classification.

### Step 1: Setup and Data Preprocessing

First, make sure you have the necessary libraries installed:

In [3]:
!pip install ipywidgets imblearn


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from ipywidgets import widgets, HBox, VBox
from IPython.display import display

1. **Data Setup:** We use a small sample dataset for demonstration purposes. In a real-world scenario, you would have a larger dataset of social media posts.

In [18]:
# Generate synthetic text data
def generate_synthetic_text_data(n_samples=1000):
    sentiments = ['positive', 'neutral', 'negative']
    text_data = []
    sentiment_labels = []
    
    for _ in range(n_samples):
        sentiment = np.random.choice(sentiments)
        if sentiment == 'positive':
            text = " ".join(["love", "great", "fantastic", "excellent", "good", "Azlina"] * np.random.randint(1, 5))
        elif sentiment == 'neutral':
            text = " ".join(["okay", "fine", "average", "mediocre", "decent"] * np.random.randint(1, 5))
        else:
            text = " ".join(["bad", "terrible", "awful", "horrible", "poor"] * np.random.randint(1, 5))
        
        text_data.append(text)
        sentiment_labels.append(sentiment)
    
    return text_data, sentiment_labels

# Generate synthetic data
texts, sentiments = generate_synthetic_text_data()

# Create a DataFrame for easier handling
df = pd.DataFrame({'text': texts, 'sentiment': sentiments})

In [19]:
df.tail()

,text,sentiment
995,okay fine average mediocre decent okay fine av...,neutral
996,love great fantastic excellent good Azlina lov...,positive
997,bad terrible awful horrible poor bad terrible ...,negative
998,okay fine average mediocre decent okay fine av...,neutral
999,love great fantastic excellent good Azlina,positive


2. **Preprocessing and Vectorization:** We use TfidfVectorizer to convert text data into numerical format, removing English stop words.

In [20]:
# Preprocess and vectorize the text data
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['text'])
y = df['sentiment']

In [21]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

3. **Model Training:** We train KNN, Naive Bayes, and SVM classifiers on the preprocessed data.

4. **Interactive UI:** Using `ipywidgets`, we create an interactive interface where users can select a model and input text to classify its sentiment.

In [22]:
# Train the classifiers
# KNN with GridSearchCV
param_grid = {'n_neighbors': [3, 5, 7, 9]}
grid = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid.fit(X_train, y_train)
print("Best parameters for KNN:", grid.best_params_)

knn = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'])
knn.fit(X_train, y_train)

Best parameters for KNN: {'n_neighbors': 3}


KNeighborsClassifier(n_neighbors=3)

In [23]:
# Gaussian Naive Bayes
nb = GaussianNB()
nb.fit(X_train.toarray(), y_train)

GaussianNB()

In [24]:
# SVM
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

SVC(kernel='linear')

In [25]:
# Create a function to classify the sentiment
def classify_sentiment(text, model):
    text_vector = vectorizer.transform([text])
    if model == nb:
        prediction = model.predict(text_vector.toarray())
    else:
        prediction = model.predict(text_vector)
    return prediction[0]

In [26]:
# Create the interactive UI
def on_button_click(b):
    model_name = model_widget.value
    text = text_widget.value
    if model_name == 'KNN':
        model = knn
    elif model_name == 'Naive Bayes':
        model = nb
    elif model_name == 'SVM':
        model = svm
    sentiment = classify_sentiment(text, model)
    output.clear_output()
    with output:
        print(f'Sentiment: {sentiment}')

In [27]:
# Define the widgets
model_widget = widgets.Dropdown(
    options=['KNN', 'Naive Bayes', 'SVM'],
    value='KNN',
    description='Model:'
)
text_widget = widgets.Text(
    value='',
    placeholder='Type something...',
    description='Text:'
)
button = widgets.Button(description="Classify")
button.on_click(on_button_click)

output = widgets.Output()

# Create the layout
ui = VBox([HBox([model_widget, text_widget, button]), output])

# Display the UI
display(ui)

In [16]:
X

<1000x30 sparse matrix of type '<class 'numpy.float64'>'
	with 9739 stored elements in Compressed Sparse Row format>